**Project NL3.14**

In [ ]:
# Mount to Google Drive 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# Imports We need in the project
import os
import re
import torch
import glob
import string
import math
import random
from termcolor import colored
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import zipfile
import seaborn as sbr
import matplotlib.pyplot as plt 
import pandas as pd
import functools
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
import enum 
from copy import copy, deepcopy

SEED = 147
torch.manual_seed(SEED)
torch.__version__

'1.7.0+cu101'

In [ ]:
!pip install --upgrade pip
!pip install transformers

from transformers import BertTokenizer, BertModel, AutoTokenizer, AdamW, BertForMaskedLM

In [ ]:
# create GPU(cuda) device 
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

**Let's see how the model works**

In [ ]:
# helper class to parse and tokenize georgian data
class GeoData:
    def __init__(self, text):
     self.txt = text
     self.sentences = []

     self.alphabet = self.get_geoalphabet()
     self.alphabet.extend(self.get_arabian_nums())
     self.alphabet.extend(self.get_romanian_nums())
     self.alphabet.extend(self.get_optional_symbols()) 
     # create pattern with our alphabet for regex
     self.pattern = re.compile(r"^[" + ''.join(self.alphabet) + r"]+$")
     # train data
     self.__process__()

    def get_alphabet(self):
      return self.alphabet

    def get_geoalphabet(self):
      alphabet = ['ა', 'ბ', 'გ', 'დ', 'ე', 'ვ', 'ზ', 'თ', 'ი', 'კ', 'ლ', 'მ', 'ნ', 'ო', 'პ', 'ჟ', 'რ', 'ს', 'ტ', 'უ', 'ფ', 'ქ', 'ღ', 'ყ', 'შ', 'ჩ', 'ც', 'ძ', 'წ', 'ჭ', 'ხ', 'ჯ', 'ჰ']
      assert len(set(alphabet)) == 33
      return alphabet 

    def get_arabian_nums(self):
      return ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    
    def get_romanian_nums(self):
      return ['X', 'I', 'V', 'L', 'C', 'D', 'M']

    def get_optional_symbols(self):
      return ['-']

    def get_badsymbols(self):
     return ['\n', '\t', '_', '+', '=', '*', '&', '%', '$', '#', '@', '^', '/', '~', '„', ',,', '”', '“']

    def get_punctuation(self):
      return ['.', ';', '!', '?', ',', ':']

    def get_endofsentence(self):
      return ['»', '«', '[', ']', '{', '}', '(', ')', '...']

    def get_sentences(self):
      return self.sentences

    def configure_word(self, w):
      w = w.strip() # remove extra white spaces
      if len(w) == 0: return False, ''
      if w.count('-') > 1: return False, '.' # check count of '-' to avoid bad words
      if w == '-' or w[0] == '-': return True, '' + w[1:] # check if word is start of dialog
      if len(w) > 15: return False, '.' # check word length to avoid coruptted words
      if (w not in self.get_alphabet() and self.pattern.match(w)) or w in [',', ':']: # because of regex syntax, check custom if words is good for our alphabet
        if w[len(w)-1] == '‐' or w[len(w)-1] == '-': # note - these two chars are different!
          return True, w[:len(w)-1] # remove no need char
        return True, w + ' ' # add white space for next word
      return False, '.' # fix end of sentence
                      
    def __process__(self):
      # clean from bad symbols
      for dl in self.get_badsymbols():
        self.txt = self.txt.replace(dl, ' ')

      # fix end of sentences and replace them with '.'
      for sw in self.get_endofsentence():
        self.txt = self.txt.replace(sw, '.')

      # split punctuation from words
      for pn in self.get_punctuation():
        self.txt = self.txt.replace(pn, ' ' + pn + ' ')
      
      # remove extra white spaces
      self.txt = re.sub(r'\s+', ' ', self.txt).strip()

      # build sentences
      sentence, length = '', 0
      for w in self.txt.split(' '):
          is_word, word = self.configure_word(w)
          
          if is_word or word in ['.', ';', '!', '?']:
            sentence, length = sentence + word, length + (1 if len(word) > 1 else 0)

          if not is_word or word in ['.', ';', '!', '?']:
            if length >= 2: self.sentences.append(sentence.strip())
            sentence, length = '', 0
      if length >= 2: self.sentences.append(sentence.strip())


In [ ]:
# Helper Class to test the model
class TestData:
  def __init__(self, text):
    self.sentences = GeoData(text).get_sentences()

  def add_data(self, text):
    self.sentences.extend(GeoData(text).get_sentences())

  def reset_data(self):
    self.sentences = []
  
  def get_testset(self):
    return [sentence.split(' ') for sentence in self.sentences]

#### usage ###
TEST = TestData('დატასეტის შექმნა და მისი მეთოდები: ედ დატა, რესეტ დატა და გეთ ტესტსეტ.')
TEST.add_data('ეს წინადადება დაემატა')
for t in TEST.get_testset():
  print(t)
print('\n')

TEST.reset_data()
TEST.add_data('წინა დატა წაიშალა და ახლა მხოლოდ ეს წინადადება დარჩება')
for t in TEST.get_testset():
  print(t)

['დატასეტის', 'შექმნა', 'და', 'მისი', 'მეთოდები', ':', 'ედ', 'დატა', ',', 'რესეტ', 'დატა', 'და', 'გეთ', 'ტესტსეტ', '.']
['ეს', 'წინადადება', 'დაემატა']


['წინა', 'დატა', 'წაიშალა', 'და', 'ახლა', 'მხოლოდ', 'ეს', 'წინადადება', 'დარჩება']


In [ ]:
# Test sentences to predict the rest of sentences with given model
sent = ['ასეთი ლამაზი ადგილი ჩემს სიცოცხლეში არსად', 
        'მე მინდა ,', 
        '', 
        'კაროჩე ესეთი ამბავია , რომ', 
        'ეს პატარა საქართველო',
        'იმიტომ',
        'ასეა თუ ისე',
        'ასეა',
        'სანამ სიკეთეა ამ ქვეყანაზედ ,',
        'აქ დიდი არჩევანი იყო ,']

to_predict = [i.split(' ') for i in sent]

In [ ]:
# load word2vec from the drive
from gensim.test.utils import get_tmpfile
fname = get_tmpfile("/content/gdrive/My Drive/NL3.14/resources/word2vec.model")
w2v = Word2Vec.load(fname)

In [ ]:
embed_dim = 300
max_seq_len = 10
vocab_size = w2v.wv.vectors.shape[0]
batch_size = 64

In [ ]:
# Base Line Model
class NGramLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLM, self).__init__()
        self.embeddings = nn.Embedding.from_pretrained(torch.tensor(w2v.wv.vectors), padding_idx=0)
        self.linear1 = nn.Linear(context_size * embedding_dim, 500)
        self.linear2 = nn.Linear(500, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = nn.functional.relu(self.linear1(embeds))
        out = self.linear2(out)
        
        return out

baseline_model = NGramLM(vocab_size, 300, 10).to(device)
baseline_model.load_state_dict(torch.load('/content/gdrive/My Drive/NL3.14/resources/baseline_model'))
baseline_model

# load the model from drive
class PredictionModel(nn.Module):
    def __init__(self, emb_dim, hid_dim, vocab_size):
        super().__init__()
        
        self.hidden_dim = hid_dim
        self.emb_dim = embed_dim
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(w2v.wv.vectors), padding_idx=0)
        self.lstm = nn.LSTM(self.emb_dim, self.hidden_dim, dropout=0.2, num_layers=2, bidirectional=True, batch_first=True)
        self.classifier = nn.Linear(4 * hid_dim, vocab_size)
              
    def forward(self, src):
        embedded = self.embedding(src)
        _, (hidden1, _) = self.lstm(embedded)
        hidden1 = torch.cat((hidden1[0], hidden1[1], hidden1[2], hidden1[3]), dim=1)
        return self.classifier(hidden1)
        
pred_model = PredictionModel(embed_dim, 128, vocab_size).to(device)
pred_model.load_state_dict(torch.load('/content/gdrive/My Drive/NL3.14/resources/prediction_model'))
pred_model

# tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
# model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased').to(device)
# model.load_state_dict(torch.load('/content/gdrive/My Drive/NL3.14/resources/bert_model'))
# model

PredictionModel(
  (embedding): Embedding(145271, 300, padding_idx=0)
  (lstm): LSTM(300, 128, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (classifier): Linear(in_features=512, out_features=145271, bias=True)
)

In [ ]:
class BertDatasetTest(torch.utils.data.Dataset):
  def __init__(self, txts):
        self.txts = txts

  def __len__(self):
        return len(self.txts)

  def __getitem__(self, index):
        sentence = self.txts[index]
        sentence.append('[MASK]')
        sentence = sentence[-max_seq_len:]
        embedX = tokenizer(' '.join(sentence),  padding='max_length', return_tensors='pt', max_length=max_seq_len, truncation=True)
        return embedX['input_ids'].to(device),embedX['token_type_ids'].to(device), embedX['attention_mask'].to(device)

class PredictionDatasetTest(torch.utils.data.Dataset):
  def __init__(self, x):
        self.x = x

  def __len__(self):
        return len(self.x)

  def __getitem__(self, index):
        def toEmbed(word):
          try:
            return w2v.wv.vocab[word].index
          except:
            try:
              return w2v.wv.vocab[w2v.wv.most_similar(word)[0][0]].index
            except:
              return random.randint(0, len(w2v.wv.vectors) - 1)
        x = [toEmbed(i) for i in self.x[index][-max_seq_len:]]
        x = (x + [0]*max_seq_len)[:max_seq_len]
        return torch.LongTensor(x).to(device)

In [ ]:
class Model(enum.Enum):
   Bert = 1
   Prediction = 2
   Baseline = 3

def BeamSearchModification(model, softmax, sentence, k, deep=4):
  Q = [(sentence, 0.0, 0)]
  punkt = [w2v.wv.vocab[i].index for i in ['.',  ',', ':']]
  eps = 10.0
  
  while Q:
    sent, prob, length = Q.pop(0)
    if length == deep: break
    topk = torch.topk(model(sent.unsqueeze(0)), dim=1, k=k)
    for i, v in zip(topk.indices.cpu().detach().numpy()[0], topk.values.cpu().detach().numpy()[0]):
      if i == sent[-1] or i in sent: continue
      if sent[-1] in punkt and i in punkt: 
        sent[-1] = i
        Q.append((sent, prob + eps * v, length))
      else:
        Q.append((torch.cat((sent, torch.tensor([i]).to(device))), prob + eps * v, length+1))
    eps -= 0.2
  return sorted(Q, key= lambda t : t[1])[0][0].cpu().detach().numpy()

In [ ]:
# display the predicted words wuth diff. color
def visualize_prediction(idx, sentence, prediction):
    sentence = ' '.join(sentence).strip()
    for pn in ['.', ';', '!', '?', ',', ':']:
        sentence = sentence.replace(' ' + pn, pn)

    prediction = ' '.join(prediction).strip()
    for pn in ['.', ';', '!', '?', ',', ':']:
        prediction = prediction.replace(' ' + pn, pn)
    
    print(colored((str(idx)+'). ' + sentence), attrs=['bold']), colored(prediction, 'magenta', attrs=['bold']))

In [ ]:
test_params = {'batch_size': batch_size,
          'shuffle': False
}

baseline_params = {'batch_size': 1,
          'shuffle': False
}

def generate(model_type, model, to_predict, n_words):
  # Set the model in evalulation mode
  model.eval()
  
  # Define the softmax function
  softmax = nn.Softmax(dim=1)
  sequences = deepcopy(to_predict)
  predicted = []

  with torch.no_grad():
    # if model_type == Model.Bert:
    #   for _ in range(n_words):
    #     bert_test_set = BertDatasetTest(sequences)
    #     bert_test_generator = torch.utils.data.DataLoader(bert_test_set, **test_params)
    #     for x_input_ids, x_token_type_ids, x_attention_mask in bert_test_generator:
    #       output = model(input_ids=x_input_ids.squeeze(1),
    #                   token_type_ids=x_token_type_ids.squeeze(1),
    #                   attention_mask=x_attention_mask.squeeze(1),
    #                 )
    #       # It is applied the softmax function to the predicted tensor
    #       prediction = softmax(output.logits)[:,-1,:]
          
    #       # It is taken the idx with the highest probability
    #       arg_max = torch.argmax(prediction, dim=1)
          
    #       # The prediction tensor is transformed into a numpy array
    #       arg_max = arg_max.cpu().detach().numpy()

    #       token = tokenizer.convert_ids_to_tokens(arg_max)
    #       print(token)

    #       full_prediction += token
    #       for f, s in zip(sequences, full_prediction):
    #         f[-1] = s
    # else:
    if model_type == Model.Baseline:
      for i in range(n_words):
        next_test_set = PredictionDatasetTest(sequences)
        next_test_generator = torch.utils.data.DataLoader(next_test_set, **baseline_params)
        preds = []
        for x in next_test_generator:
          output = baseline_model(x)
          prediction = softmax(output)
          # print(prediction.shape)
          arg_max = torch.topk(prediction.squeeze(0), dim=0, k=5).indices.cpu().detach().numpy()
          # print(arg_max)
          pred = w2v.wv.index2word[np.random.choice(arg_max)]
          preds.append(pred)
        for f, s in zip(sequences, preds):
          f.append(s)
      for f, s in zip(to_predict, sequences):
        predicted.append((f, s[len(f):]))
    else:
      for i in [4] * (n_words // 4) + ([n_words % 4] if n_words % 4 != 0 else []):
        next_test_set = PredictionDatasetTest(sequences)
        next_test_generator = torch.utils.data.DataLoader(next_test_set, **test_params)
        for x in next_test_generator:
          arg_max = []
          for _, sentence in enumerate(x):
            sentence = sentence.cpu().detach().numpy()
            arg_max.append(BeamSearchModification(pred_model, softmax, torch.tensor(sentence[sentence != 0]).to(device), 7, i))
          
          preds = [[w2v.wv.index2word[word] for word in i] for i in arg_max]
          
          for f, s in zip(sequences, preds):
            f.extend(s[-4:])
      for f, s in zip(to_predict, sequences):
        predicted.append((f, s[len(f):]))
    
  print("Prediction: \n")
  for i, s in enumerate(predicted):
    visualize_prediction(i+1, s[0], s[1])

**Enter number of words you want to predict**

In [ ]:
num_words_to_predict = 10

**Baseline model prediction**

In [ ]:
TEST.reset_data()
for i in sent:
  TEST.add_data(i)
generate(Model.Baseline, baseline_model, TEST.get_testset(), num_words_to_predict)

Prediction: 

1). ასეთი ლამაზი ადგილი ჩემს სიცოცხლეში არსად არის ეს ეს მაგრამ ეს მაგრამ:: რაც რაც
2). მე მინდა, ეს რა არის ეს ეს უნდა:: არის ეს
3). კაროჩე ესეთი ამბავია, რომ რა რა უნდა არის არ მაგრამ მაგრამ რაც მე ის
4). ეს პატარა საქართველო მაგრამ მაგრამ მაგრამ უნდა არის ეს: არ ეს რაც
5). ასეა თუ ისე უნდა მაგრამ არის: მაგრამ: არის უნდა ეს უნდა
6). სანამ სიკეთეა ამ ქვეყანაზედ, მაგრამ ეს მაგრამ::: მაგრამ მაგრამ არ უნდა
7). აქ დიდი არჩევანი იყო, ეს ეს არის მაგრამ რაც უნდა რაც: არ არის


**LSTM model prediction**

In [ ]:
TEST.reset_data()
for i in sent:
  TEST.add_data(i)
generate(Model.Prediction, pred_model, TEST.get_testset(), num_words_to_predict)

Prediction: 

1). ასეთი ლამაზი ადგილი ჩემს სიცოცხლეში არსად ჰქონდა მისთვის საკმარისი იყო მისი კარის მიმართ: მიმართ: გული სწორედ
2). მე მინდა, ახლა შემიძლია თქვენი გაბედნიერება ჩემი კეკლუცობა დედის მიმართ დედის მიმართ ქალი აყვანა
3). კაროჩე ესეთი ამბავია, რომ იქცა. ივან ფიოდოროვიჩის დარღვევა მივიჩნიოთ ეწერა: ეწერა: ხომ იმ
4). ეს პატარა საქართველო იქცა გახდა მეორე ს-ის ერთ-ერთი საუკეთესო მათგანი იყო მათგანი იყო მთავარი მომხრე
5). ასეა თუ ისე უპასუხა მოხუცმა იმ კილოთი განაგრძო მის კითხვა: კითხვა. მინდა ვიცოდე
6). სანამ სიკეთეა ამ ქვეყანაზედ, ითხოვს სამართლიანობას შეკრებაა და ის ბოროტება არ ჩავიდინო არ ჩავიდინო მათ ყოველნაირი
7). აქ დიდი არჩევანი იყო, უნდა დატოვებული არჩევანია: იმ ადამიანთა რიცხვს რომ რიცხვს რომ ოდენ რომელთა
